In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew
# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Distance loss
from geomloss import SamplesLoss

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from RegscorePy.bic import bic

# Tracking
from tqdm import trange

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
fd_derivatives = pd.read_csv("./tmp_files/fd_derivatives.csv").values
fd_u_t = pd.read_csv("./tmp_files/fd_u_t.csv").values
fd_u_t = fd_u_t.ravel()

In [4]:
est_gp = SymbolicRegressor(population_size=60000, generations=2, tournament_size=50, 
                           function_set=('add', 'sub', 'mul'), const_range=(-1, 1),
                           p_crossover=0.7, p_subtree_mutation=0.1, p_hoist_mutation=0.05,
                           p_point_mutation=0.1, parsimony_coefficient=0.001, max_samples=1.0,
                           verbose=1, low_memory=False, n_jobs=3)

est_gp.fit(fd_derivatives, fd_u_t)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    26.31         0.191359       15       0.00892383              N/A     23.68s
   1     5.56        0.0509723        7       0.00879117              N/A      0.00s


SymbolicRegressor(const_range=(-1, 1), feature_names=None,
                  function_set=('add', 'sub', 'mul'), generations=2,
                  init_depth=(2, 6), init_method='half and half',
                  low_memory=False, max_samples=1.0,
                  metric='mean absolute error', n_jobs=3, p_crossover=0.7,
                  p_hoist_mutation=0.05, p_point_mutation=0.1,
                  p_point_replace=0.05, p_subtree_mutation=0.1,
                  parsimony_coefficient=0.001, population_size=60000,
                  random_state=None, stopping_criteria=0.0, tournament_size=50,
                  verbose=1, warm_start=False)

In [6]:
# uf,u_x,u_xx,u_xxx,u_xxxx,uf*u_x,uf*u_xx,uf*u_xxx,uf*u_xxxx,u_x*u_xx,u_x*u_xxx,u_x*u_xxxx,u_xx*u_xxx,u_xx*u_xxxx,u_xxx*u_xxxx
# This is correct.
print(est_gp._program)

sub(mul(add(X2, X4), -0.766), X5)
